# Получение данных о домах города

In [1]:
import requests as rq
import pandas as pd
import json
from dadata import Dadata
from bs4 import BeautifulSoup as bs
from shapely.geometry import Polygon
from shapely.geometry import Point

In [7]:
fias_houses_codes = list(pd.DataFrame(pd.read_csv('E:/CODE/BUILDINGS_OF_DIMITROVGRAD/fias_houses_codes.csv'))['0'])

## ШАГ 0. Создаём функции

In [ ]:
print('Hello')

In [8]:
# Обработка результатов от dadata.ru
def create_line(result):
    line1 = {}
    if result != []:
        line1.update({'value':result[0]['value']})
        line1.update({'unrestricted_value':result[0]['unrestricted_value']})
        for keys in result[0]['data'].keys():
            line1.update({keys:result[0]['data'][keys]})
    return line1

In [37]:
# Получение кодов улиц
def parser(site):
    source = rq.get(site).content
    soup = bs(source, 'html.parser')

    table_label = soup.findAll('th')
    table_data = soup.findAll('td')

    table_data_list = []
    for item in list(table_data):
        table_data_list.append(item.string)
    result = {}
    for k in range(0, len(table_data_list), 6):
        result.update({table_data_list[k]:table_data_list[k+2:k+6]})

    return result

In [10]:
# Получение кодов домов
def houses_by_street(site):
    source = rq.get(site).text
    soup = bs(source)
    href = []
    for item in soup.findAll('a', class_="kladrs-objects__number"):
        attr = item.attrs
        href.append(attr['href'])
    return href

## ШАГ 1. Получаем коды улиц города

alta.ru Улицы по коду города

In [46]:
source = rq.get('https://www.alta.ru/fias/73b29372-242c-42c5-89cd-8814bc2368af/').text
soup = bs(source)
href = []
for item in soup.findAll('a', class_="jFastSearch_key"):
    attr = item.attrs
    href.append(attr['href'])
fias_streets = []
for item in href:
    fias_streets.append(item[6:-1])
del([item, soup, source, href])

## ШАГ 2. Получаем коды домов по кодам улиц

alta.ru Дома по коду улицы

In [66]:
fias_houses = []
for street_code in fias_streets:
    site = 'https://www.alta.ru/fias/' + street_code + '/'
    fias_houses.append(houses_by_street(site))

In [69]:
fias_houses_all = []
for item in fias_houses:
    for elem in item:
        fias_houses_all.append(elem)

In [71]:
fias_houses_codes = []
for item in fias_houses_all:
    fias_houses_codes.append(item[6:-1])

## ШАГ 3. Получаем данные по кодам домов

dadata.ru Поиск данных по коду дома

In [74]:
# Dadata в бесплатном пакете позволяет отработать 10 000 запросов.
# Для более крупных задач необходима подписка.
# Поэтому запросы были выполнены в разные дни, массив был разделён на две части.

token = str(open('E:/CODE/BUILDINGS_OF_DIMITROVGRAD/access_3.txt', 'r').read())
dadata = Dadata(token)

full_info = []
for i in range(len(fias_houses_codes)):
    if i >= 9000:
        code = fias_houses_codes[i]
        result = dadata.find_by_id("address", code)
        full_info.append(create_line(result))

# Another Script

In [ ]:
a = pd.read_csv('full_info.csv', encoding='cp1251')

In [ ]:
full = pd.read_csv('E:/CODE/BUILDINGS_OF_DIMITROVGRAD/full_info_edit.csv', encoding='cp1251', index_col='index', na_values=None, sep=';')

## Дом МинЖКХ

In [38]:
result_list = []
for i in range(11):
    number_page = i+1
    site = 'https://dom.mingkh.ru/ulyanovskaya-oblast/dimitrovgrad/houses?page=' + str(number_page)
    result_list.append(parser(site))

result_0 = {}
for item in result_list:
    result_0.update(item)

result_1 = pd.DataFrame(result_0).T
result_1

,0,1,2,3
1,"ул. 989 км, д. 5",166.11,1936,1
2,"ул. 981 км, д. 1",191.63,1937,1
3,"ул. 989 км, д. 3",45.75,1936,1
4,"ул. 981 км, д. 3",23.94,1937,1
5,"ул. 989 км, д. 2",92.65,1941,1
...,...,...,...,...
998,"ул. Чехова, д. 2",880.80,1975,2
999,"ул. Чкалова, д. 56",5456.98,2012,3
1000,"ул. Шмидта, д. 1",412.77,1958,1
1001,"ул. Эшенбаха, д. 15",21.00,1950,1


In [19]:
result_dict = result_1.T.to_dict()

In [82]:
addresses = {}
for i in range(len(result_dict)):
    addresses.update({list(result_dict.keys())[i]: result_dict[str(i+1)][0].split(', ')})

In [ ]:
lines = []
for el in list(addresses.values()):
    line = []
    for item in el:
        line.append(item.split('. '))
    lines.append(line)

In [112]:
street = []
for a in lines:
    street.append([a[0][1], a[1][1]])

In [123]:
data_address = pd.DataFrame(street, columns=['street', 'house'])

In [124]:
space = []
for i in range(len(result_dict)):
    space.append(result_dict[str(i+1)][1])

In [125]:
year = []
for i in range(len(result_dict)):
    year.append(result_dict[str(i+1)][2])

In [158]:
space_pd = pd.DataFrame(space, columns=['space'])
year_pd = pd.DataFrame(year, columns=['year'])

In [159]:
data_new = pd.concat([data_address, space_pd, year_pd], axis=1)

In [ ]:
list(data_new.items())[a][1]

In [ ]:
for a in range(len(list(data_new.items()))):
    for b in range(len(list(data_new.items())[a][1])):
        if list(data_new.items())[a][1][b] == 'Ленина':
            print(list(data_new.items())[0][1][b], list(data_new.items())[1][1][b], list(data_new.items())[2][1][b], list(data_new.items())[3][1][b])

## Данные OSM

In [105]:
path = 'C:/Users/User/Desktop/test.geojson'
f = json.loads(open(path, 'r', encoding='utf-8').read())
del(path)

In [1]:
#results = []
#for i in range(len(f['features'])):
#    values = []
#    for key in columns:
#        if f['features'][i]['properties'][key] != None:
#            values.append(f['features'][i]['properties'][key])
#    if values == []:
#        results.append(None)
#    else:
#        results.append(values)

In [106]:
lines = []
for i in range(len(f['features'])):
    line = {}
    keys = list(f['features'][i]['properties'].keys())[0:-1]
    for key in keys:
        line.update({key: f['features'][i]['properties'][key]})
    try:
        tags = f['features'][i]['properties']['other_tags'].split(',')
        for tag in tags:
            line.update({tag.split('=>')[0][1:-1]: tag.split('=>')[1][1:-1]})
    except:
        pass
    try:
        coords = f['features'][i]['geometry']['coordinates'][0][0]
        line.update({'coordinates': coords})
    except:
        pass
    lines.append(line)
# cleaner
del([line, key, keys, coords, i, f, tag, tags])

In [107]:
lines_pd = pd.DataFrame(lines)

In [39]:
# Short data
results = []
for i in range(len(lines_pd)):
    values = {}
    for col in lines_pd.columns:
        value = lines_pd[col][i]
        if value != None and str(value) != 'nan':
            values.update({col: value})
    results.append(values)

In [ ]:
# Optimizer Log Maker
def optimizer_log(col_name: str):
    optimized_col = {}
    i = 0
    for item in list(set(lines_pd[col_name])):
        pass

In [6]:
# Optimize Data
def optimizer(col_name: str):
    optimized_col = {}
    i = 0
    for item in list(set(lines_pd[col_name])):
        optimized_col.update({item: i})
        i += 1
    del(i)
    del(item)
    return optimized_col

### Вычисляем центроиды

In [108]:
centroids = []
for i in range(len(lines)):
    coord = []
    for item in lines[i]['coordinates']:
        coord.append(tuple([item[0], item[1]]))
    polygon = Polygon(coord)
    centroids.append(list(polygon.centroid.coords)[0])
# cleaner
del([i, coord, item, polygon])

In [177]:
lines[0]['WKT_polygon']

'POLYGON ((49.5779508 54.2384449, 49.5781548 54.2383552, 49.5782579 54.2381733, 49.5781965 54.2381359, 49.5780985 54.2380062, 49.5777832 54.238034, 49.5775619 54.2381877, 49.5775443 54.2383221, 49.5776747 54.2383677, 49.5779508 54.2384449))'

In [176]:
# Add centroids to the main data
for i in range(len(lines)):
    lines[i].update({'centroid_lon': lines[i]['centroid'][0], 'centroid_lat': lines[i]['centroid'][1]})
    lines[i].update({'WKT_centroid': str('POINT (' + str(lines[i]['centroid'][0]) + ' ' + str(lines[i]['centroid'][1]) + ')')})
    poly = 'POLYGON (('
    for j in range(len(lines[i]['coordinates'])):
        poly += str(lines[i]['coordinates'][j][0]) + ' ' + str(lines[i]['coordinates'][j][1])
        if j != len((lines)[i]['coordinates'])-1:
            poly += ', '
    poly += '))'
    lines[i].update({'WKT_polygon': poly})

In [178]:
pd.DataFrame(lines).to_csv('full_osm_data_v4.csv', encoding='utf-8', sep=';')

In [ ]:
centroids[i][0]

In [119]:
print(list(lines_pd.columns))

['osm_id', 'osm_way_id', 'name', 'type', 'aeroway', 'amenity', 'admin_level', 'barrier', 'boundary', 'building', 'craft', 'geological', 'historic', 'land_area', 'landuse', 'leisure', 'man_made', 'military', 'natural', 'office', 'place', 'shop', 'sport', 'tourism', 'coordinates', 'waterway', 'addr:housenumber', 'addr:street', 'leaf_type', 'addr:city', 'addr:country', 'addr:postcode', 'addr:region', 'building:levels', 'roof:shape', 'area:highway', 'building:colour', 'building:design', 'building:flats', 'building:material', 'start_date', 'building:part', 'building:min_level', 'industrial', 'description', 'roof:orientation', 'access', 'fee', 'parking', 'building:start_date', 'alt_name', 'surface', 'water', 'official_name', 'phone', 'full_name', 'website', 'wikidata', 'wikipedia', 'bus', 'public_transport', 'roof:material', 'roof:colour', 'healthcare', 'healthcare:speciality', 'operator', 'ref', 'vaccination', 'contact:phone', 'fence_type', 'opening_hours', 'denomination', 'religion', 'shor

In [ ]:
token = "TOKEN"
secret = "SECRET_TOKEN"
dadata = Dadata(token, secret)

In [53]:
dadata_data = []
for i in range(len(result_0)):
    result = dadata.clean("address", str('г. Димитровград, ' + result_0[str(i+1)][0]))
    dadata_data.append(result)

In [82]:
result_2 = []
for i in range(len(result_0)):
    line = {}
    line.update({'gkh_full_address': str('г. Димитровград, ' + result_0[str(i+1)][0])})
    line.update({'gkh_area': result_0[str(i+1)][1]})
    line.update({'gkh_year': result_0[str(i+1)][2]})
    line.update({'gkh_floors': result_0[str(i+1)][3]})
    result_2.append(line)

In [83]:
result_2[0]

{'gkh_full_address': 'г. Димитровград, ул. 989 км, д. 5',
 'gkh_area': '166.11',
 'gkh_year': '1936',
 'gkh_floors': '1'}

In [73]:
addresses = []
for i in range(len(result_0)):
    address = []
    for item in result_0[str(i+1)][0].split(', '):
        for item2 in item.split('. '):
            address.append(item2)
    addresses.append(address)

In [78]:
dadata_data[0]

{'source': 'г. Димитровград, ул. 989 км, д. 5',
 'result': 'Ульяновская обл, г Димитровград, ул 989 км, д 5',
 'postal_code': '433509',
 'country': 'Россия',
 'country_iso_code': 'RU',
 'federal_district': 'Приволжский',
 'region_fias_id': 'fee76045-fe22-43a4-ad58-ad99e903bd58',
 'region_kladr_id': '7300000000000',
 'region_iso_code': 'RU-ULY',
 'region_with_type': 'Ульяновская обл',
 'region_type': 'обл',
 'region_type_full': 'область',
 'region': 'Ульяновская',
 'area_fias_id': None,
 'area_kladr_id': None,
 'area_with_type': None,
 'area_type': None,
 'area_type_full': None,
 'area': None,
 'city_fias_id': '73b29372-242c-42c5-89cd-8814bc2368af',
 'city_kladr_id': '7300000200000',
 'city_with_type': 'г Димитровград',
 'city_type': 'г',
 'city_type_full': 'город',
 'city': 'Димитровград',
 'city_area': None,
 'city_district_fias_id': None,
 'city_district_kladr_id': None,
 'city_district_with_type': None,
 'city_district_type': None,
 'city_district_type_full': None,
 'city_district':

In [87]:
for i in range(len(dadata_data)):
    dadata_data[i].update(result_2[i])

In [86]:
dadata_data_copy = dadata_data

In [91]:
pd.DataFrame(dadata_data).to_csv('dadata_data.csv', encoding='cp1251')

## Join `full data` and `gkh data`

In [2]:
data_1 = pd.read_csv('E:/CODE/BUILDINGS_OF_DIMITROVGRAD/project/dadata_data.csv', sep=',', na_values=None, encoding='cp1251')
data_2 = pd.read_csv('E:/CODE/BUILDINGS_OF_DIMITROVGRAD/data_utf-8.csv', sep=';', na_values=None, low_memory=False)
data_1_json = data_1.T.to_dict()
data_2_json = data_2.T.to_dict()

In [100]:
error_data = []
joined_data = {}
for i in range(len(data_2_json)):
    joined_data.update({i: {}})

for i in range(len(data_1_json)):
    a = 0
    for j in range(len(data_2_json)):
        if a == 1:
            break
        if data_1_json[i]['house_fias_id'] == data_2_json[j]['house_fias_id']:
            joined_data[j].update(data_1_json[i])
            a += 1
        joined_data[j].update(data_2_json[j])
    if a != 1:
        error_data.append(data_1_json[i])

# FIAS

## FIAS | BeautifulSoup

In [1]:
from bs4 import BeautifulSoup as bs

In [2]:
soup = bs(open('E:/opendata/ФИАС/73/AS_HOUSES_20211125_6f6cc0e9-6978-459e-9755-0deed43b46ef.XML', 'r', encoding='utf-8'), 'xml')

In [ ]:
soup.find('HOUSE')

In [3]:
count = 0
for item in soup.find_all('HOUSE'):
    if item.attrs['ISACTUAL'] == '1':
        count += 1
print(count)

300092


In [3]:
data = []
houses = list(soup.findAll('HOUSE'))
for house in houses:
    data.append(house.attrs)

In [ ]:
import pandas as pd
pd.DataFrame(data).to_csv('houses.csv')

In [ ]:
print('hello')

In [2]:
city_general_data = {
    'name_ru': 'Димитровград',
    'name_en': 'Dimitrovgrad',
    'region_full_name_ru': 'Ульяновская область',
    'region_full_name_en': 'Ulyanovsk region',
    'fias_code_city': '73b29372-242c-42c5-89cd-8814bc2368af',
    'fias_code_region': 'fee76045-fe22-43a4-ad58-ad99e903bd58'
}

In [3]:
city_general_data

{'name_ru': 'Димитровград',
 'name_en': 'Dimitrovgrad',
 'region_full_name_ru': 'Ульяновская область',
 'region_full_name_en': 'Ulyanovsk region',
 'fias_code_city': '73b29372-242c-42c5-89cd-8814bc2368af',
 'fias_code_region': 'fee76045-fe22-43a4-ad58-ad99e903bd58'}

## FIAS | lxml

In [67]:
fias_desc = open('E:/opendata/ФИАС/AS_ADDR_OBJ_TYPES_20211125_402d7dfd-7209-4f9e-8fbe-021ef197d1cc.XML', 'r', encoding='utf-8').read()

In [68]:
root = etree.fromstring(fias_desc)

In [40]:
data = []
for i in range(len(root.findall('HOUSETYPE'))):
    data.append(root.findall('HOUSETYPE')[i].items())

In [47]:
pd.DataFrame(data)

,0,1,2,3,4,5,6,7
0,"(ID, 1)","(NAME, Корпус)","(SHORTNAME, к.)","(DESC, Корпус)","(ISACTIVE, true)","(UPDATEDATE, 2015-12-25)","(STARTDATE, 2015-12-25)","(ENDDATE, 2079-06-06)"
1,"(ID, 2)","(NAME, Строение)","(SHORTNAME, стр.)","(DESC, Строение)","(ISACTIVE, true)","(UPDATEDATE, 2015-12-25)","(STARTDATE, 2015-12-25)","(ENDDATE, 2079-06-06)"
2,"(ID, 3)","(NAME, Сооружение)","(SHORTNAME, соор.)","(DESC, Сооружение)","(ISACTIVE, true)","(UPDATEDATE, 2015-12-25)","(STARTDATE, 2015-12-25)","(ENDDATE, 2079-06-06)"
3,"(ID, 4)","(NAME, Литера)","(SHORTNAME, литера)","(DESC, Литера)","(ISACTIVE, false)","(UPDATEDATE, 1900-01-01)","(STARTDATE, 1900-01-01)","(ENDDATE, 2015-12-25)"


In [51]:
root.findall('HOUSETYPE')[i].items()[0]

('ID', '4')

In [73]:
result = []
for item in root.getchildren():
    line = {}
    for el in item.items():
        line.update({el[0]: el[1]})
    result.append(line)

In [84]:
pd.DataFrame(result)

,ID,LEVEL,NAME,SHORTNAME,DESC,UPDATEDATE,STARTDATE,ENDDATE,ISACTIVE
0,2,1,Автономная область,а.обл.,Автономная область,2015-11-05,2015-11-05,2079-06-06,true
1,5,1,Автономная область,Аобл,Автономная область,1900-01-01,1900-01-01,2015-11-05,true
2,4,1,Автономный округ,АО,Автономный округ,1900-01-01,1900-01-01,2015-11-05,true
3,3,1,Автономный округ,а.окр.,Автономный округ,2015-11-05,2015-11-05,2079-06-06,true
4,6,1,Город,г,Город,1900-01-01,1900-01-01,2015-11-05,true
...,...,...,...,...,...,...,...,...,...
391,410,16,Участок,уч-к,Участок,1900-01-01,1900-01-01,2015-12-31,true
392,411,16,Ферма,ферма,Ферма,1900-01-01,1900-01-01,2015-12-31,true
393,412,16,Хутор,х,Хутор,1900-01-01,1900-01-01,2079-06-06,true
394,413,16,Шоссе,ш,Шоссе,1900-01-01,1900-01-01,2079-06-06,true


In [1]:
import pymongo as pm

In [9]:
client = pm.MongoClient('localhost', 27017)

In [3]:
BoD = client.get_database("BoD")

In [4]:
data_ADDR_OBJ = BoD.get_collection("data_ADDR_OBJ")
data_ADDR_OBJ_PARAMS = BoD.get_collection("data_ADDR_OBJ_PARAMS")
PARAM_TYPES = BoD.get_collection("PARAM_TYPES")
data_MUN_HIERARCHY = BoD.get_collection("data_MUN_HIERARCHY")

In [ ]:
data_ADDR_OBJ.find_one()

In [ ]:
for i in range(len(list([*data_ADDR_OBJ_PARAMS.find({'OBJECTID': '95250250'})]))):
    print(PARAM_TYPES.find_one({'ID': list([*data_ADDR_OBJ_PARAMS.find({'OBJECTID': '95250250'})])[i]['TYPEID']})['NAME'])

In [103]:
child_of_dim = [*data_MUN_HIERARCHY.find({'PARENTOBJID': '1325470', 'ISACTIVE': '1'})]

In [130]:
all_child = []
for i in range(len(child_of_dim)):
    level = 6
    while level < 9:
        data = list(data_ADDR_OBJ.find({'OBJECTID': child_of_dim[i]['OBJECTID'], 'LEVEL': str(level)}))
        if data == []:
            pass
        else:
            for item in data:
                all_child.append(item)
        level += 1

In [131]:
all_child[0]

{'_id': ObjectId('61e59f579e54b96ad4a66179'),
 'ID': '1642045',
 'OBJECTID': '1329563',
 'OBJECTGUID': '469a960e-e8dd-493f-8ce9-bab617f55069',
 'CHANGEID': '3640235',
 'NAME': 'Прибой',
 'TYPENAME': 'тер. СНТ',
 'LEVEL': '7',
 'OPERTYPEID': '20',
 'PREVID': '1642014',
 'NEXTID': '0',
 'UPDATEDATE': '2019-06-17',
 'STARTDATE': '2019-06-17',
 'ENDDATE': '2079-06-06',
 'ISACTUAL': '1',
 'ISACTIVE': '1'}